In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime 
from pathlib import Path 
import os
import sys 

print (sys.version)

base_dir = Path ('C:/Users/Frederic/Documents/kaggle/earthquake/')
all_files = base_dir.glob ('*')
for filename in all_files :
    print(filename)

print (datetime.datetime.now())        

3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]
C:\Users\Frederic\Documents\kaggle\earthquake\.ipynb_checkpoints
C:\Users\Frederic\Documents\kaggle\earthquake\earthquake predict.ipynb
C:\Users\Frederic\Documents\kaggle\earthquake\LinearDiscriminantAnalysis-lsqr-auto.csv
C:\Users\Frederic\Documents\kaggle\earthquake\randomTreeSubmission_ 12-none-50-70 RS 33 rerun on X.csv
C:\Users\Frederic\Documents\kaggle\earthquake\randomTreeSubmission_ 6-log2-50-80 RS 33 reduce feature X.csv
C:\Users\Frederic\Documents\kaggle\earthquake\sample_submission.csv
C:\Users\Frederic\Documents\kaggle\earthquake\test.csv
C:\Users\Frederic\Documents\kaggle\earthquake\train.csv
2024-04-16 15:35:39.450018


In [2]:
data = pd.read_csv (base_dir / Path ("train.csv"))
print (data.dtypes)

building_id                                int64
count_floors_pre_eq                        int64
age                                        int64
area_percentage                            int64
height_percentage                          int64
land_surface_condition                    object
foundation_type                           object
roof_type                                 object
ground_floor_type                         object
other_floor_type                          object
position                                  object
plan_configuration                        object
has_superstructure_adobe_mud               int64
has_superstructure_mud_mortar_stone        int64
has_superstructure_stone_flag              int64
has_superstructure_cement_mortar_stone     int64
has_superstructure_mud_mortar_brick        int64
has_superstructure_cement_mortar_brick     int64
has_superstructure_timber                  int64
has_superstructure_bamboo                  int64
has_superstructure_r

In [3]:
features_cat = ['land_surface_condition','foundation_type','roof_type','ground_floor_type','other_floor_type','position',
                'plan_configuration', 'legal_ownership_status']
features_bool = [ 'has_superstructure_adobe_mud','has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
                  'has_superstructure_cement_mortar_stone','has_superstructure_mud_mortar_brick','has_superstructure_cement_mortar_brick',
                  'has_superstructure_timber', 'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered', 
                  'has_superstructure_rc_engineered','has_superstructure_other','has_secondary_use', 'has_secondary_use_agriculture', 
                  'has_secondary_use_hotel', 'has_secondary_use_rental', 'has_secondary_use_institution', 'has_secondary_use_school',
                  'has_secondary_use_industry', 'has_secondary_use_health_post', 'has_secondary_use_gov_office',
                  'has_secondary_use_use_police','has_secondary_use_other'] 
features_num =    ['count_floors_pre_eq','age', 'area_percentage', 'height_percentage', 'count_families']

mean_damage = data.damage_grade.mean()
print (f"overall mean damage {mean_damage}")


for f in features_bool :
        d = data [['damage_grade', f]].corr(method= 'spearman')
        # print (f"correlation  {f} and damage  : ")
        # print (d)
        d = data [['building_id','damage_grade', f]]
        grouped =  d.groupby (by = f)
        print (grouped.agg ( {"damage_grade" : ["mean"]}))
    

for f in features_cat : 
    # d = data [['damage_grade', f]].corr(method= 'spearman')
    # print (f"correlation  {f} and damage  : ")
    # print (d)
    
    d = data [['building_id','damage_grade', f]]
    grouped =  d.groupby (by = f)
    print (grouped.agg ( {"damage_grade" : ["mean"]}))
    
for f in features_num :
    print (f"average for feature {f} is {data[f].mean()}")
    d = data [['building_id','damage_grade', f]]
    grouped = d.groupby (by = 'damage_grade')
    print  (grouped.agg ({f : ['mean']}))

overall mean damage 2.1435
                             damage_grade
                                     mean
has_superstructure_adobe_mud             
0                                2.129681
1                                2.260664
                                    damage_grade
                                            mean
has_superstructure_mud_mortar_stone             
0                                       1.850722
1                                       2.400282
                              damage_grade
                                      mean
has_superstructure_stone_flag             
0                                  2.13818
1                                  2.31405
                                       damage_grade
                                               mean
has_superstructure_cement_mortar_stone             
0                                          2.184883
1                                          1.809524
                                    damage_

In [4]:
dd = data [['has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered','damage_grade', 'building_id']]

display (data [(data ['has_superstructure_rc_non_engineered'] == 1) & (data ['has_superstructure_rc_engineered'] == 1)].head())

dd_true = data [(data['has_superstructure_rc_non_engineered'] == 1) & (data['has_superstructure_rc_engineered'] == 1)] 

print (dd_true ['damage_grade'].value_counts())


,building_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
2,2,2,0,12,6,t,i,x,v,s,...,0,0,0,0,0,0,0,0,0,1
28,28,1,0,14,3,t,i,x,v,j,...,0,0,0,0,0,0,0,0,0,1
251,251,4,0,18,6,t,i,x,v,s,...,0,0,0,0,0,0,0,0,0,1
411,411,2,0,6,7,t,i,x,x,s,...,0,0,0,0,0,0,0,1,0,2
454,454,1,0,18,3,t,u,n,v,j,...,1,0,0,0,0,0,0,0,0,1


damage_grade
1    35
2     3
Name: count, dtype: int64


In [6]:
best_features = [ 'plan_configuration_a', 'plan_configuration_c', 
                 'plan_configuration_m', 'plan_configuration_n', 'plan_configuration_o', 'other_floor_type_s',
                 'ground_floor_type_v', 'roof_type_x', 'foundation_type_i', 'has_superstructure_rc_engineered',
                 'has_secondary_use_hotel', 'has_superstructure_rc_non_engineered', 'has_secondary_use_institution']

data_features = data.drop (columns = ['damage_grade', 'building_id'])
y = data ['damage_grade']

In [27]:
#$ !pip install category-encoders 
from sklearn.preprocessing import StandardScaler

#from category_encoders.ordinal import OrdinalEncoder
#from category_encoders.woe import WOEEncoder
#from category_encoders.target_encoder import TargetEncoder
#from category_encoders.sum_coding import SumEncoder
#from category_encoders.m_estimate import MEstimateEncoder
#from category_encoders.leave_one_out import LeaveOneOutEncoder
#from category_encoders.helmert import HelmertEncoder
#from category_encoders.cat_boost import CatBoostEncoder
#from category_encoders.james_stein import JamesSteinEncoder
#from category_encoders.one_hot import OneHotEncoder
#TE_encoder = TargetEncoder([features_cat])
# cat_te = 0 


# prepare input data
def convert_cat_inputs(X):
    result = pd.DataFrame ()
    for cat_col in X.columns :
        values = sorted(list (X[cat_col].unique()))
        for v in values : 
            result [cat_col + '_' + str (v)] =  np.where (X [cat_col] == v, 1, 0)
    return result 

def normalize_int_inputs(X):
    result = pd.DataFrame ()
    for int_col in X.columns :
        m = X[int_col].max()
        n = X[int_col].min()
        result [int_col] =  (X [int_col] - n)/ (m-n)
    return result

def custom_normalize_numbers (X) :
    result  = X
    result ['count_floors_pre_eq' ] = X ['count_floors_pre_eq'] / 5 
    result ['age' ] = X ['age'] / 100 
    result ['area_percentage' ] = X ['area_percentage'] / 100
    result ['height_percentage' ] = X ['height_percentage'] / 100
    
    
def custom_normalize (data, y = None) :
#    global cat_te
#    if (y is not None) :
#      print ("execute fit transform")  
#      cat_te = TE_encoder.fit_transform(data [features_cat], y)
#    else :
#      print ("execute fit only")  
#      cat_te = TE_encoder.transform (data [features_cat])    
#    my_cat_hot = convert_cat_inputs (data[features_cat])
    cat_hot = pd.get_dummies (data[features_cat], dtype ='int')
#    scaler = StandardScaler ()
    result1 = data [features_bool]
    result2  =  normalize_int_inputs (data[features_num])
    
    print ('standard scaler executed')
    
    result   = pd.concat  ([result1, result2, cat_hot], axis = 1) 
    # X_no_cat = custom_normalize_numbers (X_no_cat) 
    
    return  result 

X = custom_normalize (data_features)

# X = X[best_features]

print (f"shape of X : {X.shape}" )

standard scaler executed
shape of X : (4000, 65)


In [8]:
display (X.head)

<bound method NDFrame.head of       plan_configuration_a  plan_configuration_c  plan_configuration_m  \
0                        0                     0                     0   
1                        0                     1                     0   
2                        0                     0                     0   
3                        0                     0                     0   
4                        0                     0                     0   
...                    ...                   ...                   ...   
3995                     0                     0                     0   
3996                     0                     0                     0   
3997                     0                     0                     0   
3998                     0                     0                     0   
3999                     0                     0                     0   

      plan_configuration_n  plan_configuration_o  other_floor_type_s  \
0        

In [56]:
# create the data sets for training 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 3)

print (f"samples X : {X.shape[0]}  size y : {y.shape [0] } size X_train : {X_train.shape[0]},  size y_train : {y_train.shape[0], }")
print ('X, y data split complete' )


samples X : 4000  size y : 4000 size X_train : 3200,  size y_train : (3200,)
X, y data split complete


In [10]:
# general comparison tools 

from sklearn.metrics  import classification_report
from sklearn.model_selection import GridSearchCV

def custom_compare (y_pred, y_test) :
    if (y_pred.shape != y_test.shape) :
      print (f"shapes don't match. y_pred : {y_pred.shape} y_test : {y_test.shape}")
      return
    compare = pd.DataFrame ()
    compare ['predict'] =  y_pred.T
    compare ['test'] =  list (y_test) 
    compare ['correct'] = (compare ['predict'] == compare ['test'])

    print (compare.head())
    print (classification_report (y_pred, y_test))
    
    diff = sum (compare ['correct'])
    print (f"correct prediction in {diff} cases. this is {diff/compare.shape [0]} precentage" )
    return diff 

In [57]:
# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
param_grid = { 
	'n_estimators': [ 60, 70, 80, 90, 100, 110 ], 
	'max_features': ['sqrt', 'log2', None], 
	'max_depth': [ 3, 6, 9, 12], 
	'max_leaf_nodes': [  10, 15, 20, 25, 30, 35, 40, 45, 50], 
} 
grid_search = GridSearchCV(RandomForestClassifier(), 
						param_grid=param_grid) 
grid_search.fit(X_train, y_train) 
print(grid_search.best_estimator_) 

RandomForestClassifier(max_depth=12, max_leaf_nodes=10, n_estimators=110)


In [89]:
classifier = RandomForestClassifier(max_depth=12, max_features=None, 
                       n_estimators=253, random_state = 7)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
custom_compare (y_pred, y_test)

   predict  test  correct
0        2     3    False
1        3     3     True
2        2     3    False
3        2     2     True
4        2     2     True
              precision    recall  f1-score   support

           1       0.59      0.72      0.65       116
           2       0.74      0.58      0.65       511
           3       0.36      0.54      0.43       173

    accuracy                           0.59       800
   macro avg       0.56      0.61      0.58       800
weighted avg       0.64      0.59      0.60       800

correct prediction in 471 cases. this is 0.58875 precentage


471

In [90]:
df_y_pred = pd.DataFrame (y_pred)
print ('predicted values')
print (df_y_pred [0].value_counts())
print ('expected values')
print (y_test.value_counts())
print ('training data' )
print (y_train.value_counts())


predicted values
0
2    511
3    173
1    116
Name: count, dtype: int64
expected values
damage_grade
2    396
3    261
1    143
Name: count, dtype: int64
training data
damage_grade
2    1572
3    1042
1     586
Name: count, dtype: int64


In [91]:

# first apply to all data 

cfg.fit(X_test, y_test) 

data_submit = pd.read_csv (base_dir /  Path ('test.csv'))

data_submit1 = data_submit.drop (columns = ['building_id'])

X_submit = custom_normalize (data_submit1)

y_pred = pd.DataFrame (cfg.predict(X_submit))

for_submission = y_pred.join (data_submit ['building_id'] )
for_submission = for_submission.rename (columns = {0 : 'damage_grade'}) 

print (for_submission.damage_grade.value_counts())

display (for_submission)


for_submission.to_csv (base_dir / Path  ("randomTreeSubmission_ 12-None---253 RS 33 add Test.csv"), index = False)

standard scaler executed
damage_grade
2    617
3    212
1    171
Name: count, dtype: int64


,damage_grade,building_id
0,2,0
1,1,1
2,1,2
3,3,3
4,1,4
...,...,...
995,1,995
996,2,996
997,3,997
998,2,998


In [45]:
from sklearn.ensemble import GradientBoostingClassifier
param_grid = { 
	'n_estimators': [ 90, 105, 115, 125, 135 ],
    'max_depth'   : [3,6,9,12,15,18], 
	'max_leaf_nodes': [  15, 20, 25, 30], 
} 
grid_search = GridSearchCV(GradientBoostingClassifier(), 
						param_grid=param_grid) 
grid_search.fit(X_train, y_train) 
print(grid_search.best_estimator_) 

GradientBoostingClassifier(max_leaf_nodes=15, n_estimators=90)


In [114]:
cfg =  GradientBoostingClassifier(max_depth=12, 
								max_leaf_nodes= 25,
									random_state = 3, 
									n_estimators=60,
                                    subsample = 0.75,
                                    n_iter_no_change = 5, 
                                    warm_start = True) 
cfg.fit(X_train, y_train) 
y_predict = cfg.predict(X_test) 
custom_compare(y_predict, y_test)

   predict  test  correct
0        2     3    False
1        3     3     True
2        3     3     True
3        2     2     True
4        2     2     True
              precision    recall  f1-score   support

           1       0.57      0.72      0.64       112
           2       0.77      0.57      0.65       535
           3       0.30      0.52      0.38       153

    accuracy                           0.58       800
   macro avg       0.55      0.60      0.56       800
weighted avg       0.65      0.58      0.60       800

correct prediction in 464 cases. this is 0.58 precentage


464

In [115]:
cfg.fit(X_test, y_test) 

y_pred = pd.DataFrame (cfg.predict(X_submit))

for_submission = y_pred.join (data_submit ['building_id'] )
for_submission = for_submission.rename (columns = {0 : 'damage_grade'}) 

display (for_submission)

for_submission.to_csv (base_dir / Path  ("gradient boost 12-25-60-075-5 add Test warm.csv"), index = False)

,damage_grade,building_id
0,2,0
1,1,1
2,1,2
3,2,3
4,2,4
...,...,...
995,2,995
996,3,996
997,3,997
998,2,998


In [50]:
from sklearn.ensemble import AdaBoostClassifier

# display (X_train)

   
param_grid = { 
    'n_estimators': [110,115,120,125,130, 140, 150],
    
} 
grid_search = GridSearchCV(AdaBoostClassifier(), 
                        param_grid=param_grid) 
grid_search.fit(X_train, y_train) 
print(grid_search.best_estimator_)  

AdaBoostClassifier(n_estimators=125)


In [51]:
cfg = AdaBoostClassifier(n_estimators = 125)
cfg.fit(X_train, y_train) 
y_predict = cfg.predict(X_test) 
custom_compare(y_predict, y_test)

   predict  test  correct
0        2     2     True
1        2     2     True
2        3     3     True
3        3     3     True
4        1     2    False
              precision    recall  f1-score   support

           1       0.55      0.63      0.58       131
           2       0.57      0.56      0.57       409
           3       0.48      0.46      0.47       260

    accuracy                           0.54       800
   macro avg       0.53      0.55      0.54       800
weighted avg       0.54      0.54      0.54       800

correct prediction in 431 cases. this is 0.53875 precentage


431

In [52]:
# cfg = AdaBoostClassifier(n_estimators = 110)
cfg.fit(X_test, y_test) 

y_pred = pd.DataFrame (cfg.predict(X_submit))

for_submission = y_pred.join (data_submit ['building_id'] )
for_submission = for_submission.rename (columns = {0 : 'damage_grade'}) 

display (for_submission)

for_submission.to_csv (base_dir / Path  ("Ada_boost 125.csv"), index = False)

,damage_grade,building_id
0,2,0
1,1,1
2,1,2
3,3,3
4,2,4
...,...,...
995,2,995
996,3,996
997,2,997
998,2,998


In [ ]:
!pip install lazypredict
from lazypredict.Supervised import LazyClassifier


clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

print (type(models))
print(models)


In [42]:
from sklearn.naive_bayes import BernoulliNB
# alpha : float or array-like of shape (n_features,), default=1.0 Additive (Laplace/Lidstone) smoothing parameter (set alpha=0 and force_alpha=True, for no smoothing).
#force_alpha: bool, default=True
# If False and alpha is less than 1e-10, it will set alpha to 1e-10. If True, alpha will remain unchanged. This may cause numerical errors if alpha is too close to 0.
# binarize : float or None, default=0.0
# Threshold for binarizing (mapping to booleans) of sample features. If None, input is presumed to already consist of binary vectors.
# fit_prior bool, default=True
# Whether to learn class prior probabilities or not. If false, a uniform prior will be used.
# class_prior array-like of shape (n_classes,), default=None
# Prior probabilities of the classes. If specified, the priors are not adjusted according to the data.


clf = BernoulliNB()


clf.fit(X_train, y_train) 
y_clf = clf.predict(X_test) 
custom_compare (y_clf, y_test)


   predict  test  correct
0        3     2    False
1        2     2     True
2        3     3     True
3        3     3     True
4        1     2    False
              precision    recall  f1-score   support

           1       0.64      0.62      0.63       156
           2       0.34      0.57      0.42       242
           3       0.72      0.44      0.55       402

    accuracy                           0.51       800
   macro avg       0.57      0.54      0.53       800
weighted avg       0.59      0.51      0.52       800

correct prediction in 410 cases. this is 0.5125 precentage


410

In [54]:
cfg = LinearDiscriminantAnalysis()
cfg.fit(X_train, y_train) 
y_predict = cfg.predict(X_test) 
custom_compare (y_predict, y_test)

   predict  test  correct
0        2     2     True
1        2     2     True
2        2     3    False
3        2     3    False
4        1     2    False
              precision    recall  f1-score   support

           1       0.54      0.67      0.60       121
           2       0.66      0.57      0.61       471
           3       0.40      0.48      0.44       208

    accuracy                           0.56       800
   macro avg       0.53      0.57      0.55       800
weighted avg       0.58      0.56      0.56       800

correct prediction in 447 cases. this is 0.55875 precentage


447

In [55]:

cfg.fit(X_test, y_test) 

y_pred = pd.DataFrame (cfg.predict(X_submit))

for_submission = y_pred.join (data_submit ['building_id'] )
for_submission = for_submission.rename (columns = {0 : 'damage_grade'}) 

display (for_submission)

for_submission.to_csv (base_dir / Path( "LinearDiscriminantAnalysis-lsqr-auto with test.csv"), index = False)

,damage_grade,building_id
0,2,0
1,1,1
2,1,2
3,2,3
4,2,4
...,...,...
995,1,995
996,2,996
997,2,997
998,2,998
